Check Python version

In [ ]:
!python --version

Import libraries

In [ ]:
import json
import yaml
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

### **Extract Data**

In [ ]:
# Local directory
data_dir = '../datasets/climate'
file_name = 'temp_anom_w_forcing'
#data_dir = '../datasets/finance'
#file_name = 'Returns_short_interest_data_train'
file_path = f'{data_dir}/{file_name}.csv'

In [ ]:
dataset = pd.read_csv(file_path)

In [ ]:
dataset.shape

In [ ]:
dataset.columns

In [ ]:
dataset.head()

In [ ]:
dataset.tail()

Define the required time and target columns

In [ ]:
time_col = dataset.columns[0]
target_col = dataset.columns[-1]
dataset[time_col] = dataset[time_col].astype(str)

Change target column to float

In [ ]:
# dataset[target_col] = dataset[target_col].astype(float)

In [ ]:
# dataset.head()

Split dataset into train and test sets

In [ ]:
train_size = 0.8
train_points = int(train_size*len(dataset))
train_df = dataset.iloc[:train_points]
test_df = dataset.iloc[train_points:]

Plot graph

In [ ]:
figsize = (16, 8)
train_df.set_index(time_col)[target_col].plot(figsize=figsize);

### **Train API**

Train data: convert df to list-of-list

In [ ]:
train_data = []
for value in train_df.values:
    train_data.append(list(value))

In [ ]:
train_data[:2]

Custom Model Request (optional)

In [ ]:
# # # Default model request
# # model_request = {
# #     'type': 'meta_lr',  # 'meta_wa'
# #     'scorers': ['smape', 'mape'],
# #     'params': {
# #         'preprocessors': [
# #             {'type': 'dartsimputer', 'params': {'fill': 'auto'}},
# #             # {'type': 'simpleimputer', 'params': {'strategy': 'mean'}},
# #             {'type': 'minmaxscaler'},
# #         ],
# #         'base_models': [
# #             {'type': 'darts_rnn'},
# #             {'type': 'darts_lightgbm'},
# #             {'type': 'darts_autotheta'},
# #             {'type': 'darts_autoarima'},
# #             {'type': 'darts_autoets'},
# #             # {'type': 'stats_autotheta'},
# #             # {'type': 'stats_autoarima'},
# #             # {'type': 'stats_autoets'},
# #         ],
# #     },
# # }

# # Onboard NeuralProphet customized model request
# np_external_model_request = {
#     'type': 'neuralprophet',
#     'metrics': ['rmse', 'mae'],
#     # 'metrics': ['smape', 'mape'],
#     'params': {
#         'lagged_regressors': [
#             {'index': 0},
#             {'index': 1},
#             {'index': 2},
#             {'index': 3},
#             {'index': 4},
#             {'index': 5},
#             {'index': 6},
#             {'index': 7},
#             {'index': 8}
#         ],
#         "epochs": 5
#     },
# }
# # np_external_model_request = None


# # Customized model request
# model_request = {
#     'type': 'meta_lr',  # 'meta_wa'
#     'scorers': ['smape', 'mape'],
#     'params': {
#         'preprocessors': [
#             {'type': 'dartsimputer', 'params': {'fill': 'auto'}},
#             # {'type': 'simpleimputer', 'params': {'strategy': 'mean'}},
#             {'type': 'minmaxscaler'},
#         ],
#         'base_models': [
#             {'type': 'darts_naive'},
#             {'type': 'darts_seasonalnaive'},
#             {'type': 'darts_autotheta'},
#             # {'type': 'stats_autotheta'},
#             {'type': 'darts_autoets'},
#             # {'type': 'stats_autoets'},
#             {'type': 'darts_autoarima'},
#             # {'type': 'stats_autoarima'},
#             {'type': 'darts_tbats'},
#             # {'type': 'darts_linearregression'},
#             {'type': 'darts_lightgbm',
#              'params': {
#                 'lags': 12,
#                 'lags_future_covariates': [0, 1, 2],
#                 'output_chunk_length': 6,
#                 'verbose': -1
#             }},  #'lags_past_covariates' 
#             {'type': 'darts_rnn',
#              'params': {
#                 'model': 'LSTM',
#                 'hidden_dim': 10,
#                 'n_rnn_layers': 3
#             }},
#             {'type': 'neuralprophet', 
#              'external_params': np_external_model_request
#             }  # Onboard NeuralProphet external service
#         ],
#     },
# }

This is for JSON model_request

In [ ]:
# file_path = 'model_request.json'
# # For writing the model request to a json file
# with open(file_path, 'r') as file:
#     model_request = json.load(file)

This is for YAML model_request

In [ ]:
file_path = 'model_request.yaml'
# For reading the model request from a yaml file
with open(file_path, 'r') as file:
    model_request = yaml.safe_load(file)

Train API JSON Payload

In [ ]:
api_json = {
    'data': train_data,
    'model': model_request  # (optional) can be commented out
}

Build URL

In [ ]:
with open('url.yaml', 'r') as file:
    url_dict = yaml.safe_load(file)

In [ ]:
# URL to our SYBIL AWS service
protocol = url_dict['protocol']
host = url_dict['host']
port = url_dict['port']
endpoint = 'train'

url = '%s://%s:%s/%s' % (protocol, host, str(port), endpoint)

In [ ]:
# protocol, host, port

Call endpoint, receive response JSON, write to output file

In [23]:
%%time
response = requests.post(url, json=api_json)
print(response)
print()

In [ ]:
train_json_out = response.json()  # dict output
train_json_out

### **Forecast API**

Forecast Dates: predict the next 32 years of temperature anomaly data (1980-2012)

In [ ]:
test_data = []
for value in test_df.drop(columns=target_col).values:
    test_data.append(list(value))

In [ ]:
test_data[:2]

Model from Train API

In [ ]:
model = train_json_out['model']

Forecast API JSON Payload

In [ ]:
api_json = {
    'model': model,
    'data': test_data
}

Build URL (same as train except for the endpoint)

In [ ]:
endpoint = 'forecast'

url = '%s://%s:%s/%s' % (protocol, host, str(port), endpoint)

Call endpoint, receive response JSON, write to output file

In [ ]:
%%time
response = requests.post(url, json=api_json)
print(response)
print()

In [ ]:
forecast_json_out = response.json()  # dict output
forecast_json_out

Combine forecast dates and output

In [ ]:
forecast_df = pd.DataFrame(
    data=forecast_json_out['data'],
    columns=[time_col, target_col],
)

In [ ]:
forecast_df.shape

In [ ]:
forecast_df.columns

In [ ]:
forecast_df.head()

In [ ]:
forecast_df.tail()

Plot train and forecast side-by-side

In [ ]:
train_df['color'] = 'b'
train_df.set_index(time_col)[target_col].plot(figsize=figsize, color=train_df['color']);

In [ ]:
forecast_df['color'] = 'r'
forecast_df.set_index(time_col).plot(figsize=figsize, color=forecast_df['color']);

Combine train and forecast sets

In [ ]:
df = pd.concat([train_df, forecast_df]).reset_index(drop=True)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.tail()

Plot combined train and forecast sets with the vertical black line as the cut-off

In [ ]:
df.set_index(time_col)[target_col].plot(figsize=figsize, color='r')
plt.axvline(x=len(train_df), color='black', label='Train/Forecast set cut-off')
plt.text(x=len(train_df)-9, y=forecast_df[target_col].max(), s='Train', fontweight='bold', fontsize=14)
plt.text(x=len(train_df)+1, y=forecast_df[target_col].max(), s='Forecast', fontweight='bold', fontsize=14)
plt.ylabel(ylabel='Temperature Anomaly (' + u'\N{DEGREE SIGN}' + 'C)')
plt.title(label='Annual Temperature Anomaly (Train: 1850-1979) (Forecast: 1980-2012)', fontweight='bold', fontsize=20)
plt.show();

Plot original dataset to compare

In [ ]:
figsize = (16, 8)
dataset.set_index(time_col)[target_col].plot(figsize=figsize);
plt.ylabel(ylabel='Temperature Anomaly (' + u'\N{DEGREE SIGN}' + 'C)')
plt.title(label='Annual Temperature Anomaly (1850-2012)', fontweight='bold', fontsize=20)
plt.show();

Plot original dataset (blue) with SYBIL forecasted values (red)

In [ ]:
df.set_index(time_col)[target_col].plot(figsize=figsize, color='r')
dataset.set_index(time_col)[target_col].plot(figsize=figsize);
plt.axvline(x=len(train_df), color='black', label='Train/Forecast set cut-off')
plt.text(x=len(train_df)-9, y=forecast_df[target_col].max(), s='Train', fontweight='bold', fontsize=14)
plt.text(x=len(train_df)+1, y=forecast_df[target_col].max(), s='Forecast', fontweight='bold', fontsize=14)
plt.ylabel(ylabel='Temperature Anomaly (' + u'\N{DEGREE SIGN}' + 'C)')
plt.title(label='Annual Temperature Anomaly (Train: 1850-1979) (Forecast: 1980-2012)', fontweight='bold', fontsize=20)
plt.show();